In [7]:
import os
import pandas as pd
import torch
import csv
import numpy as np
import numpy.matlib
import codecs as cs
import scipy.io as sio
from torch.utils.data import DataLoader

In [8]:
humanact12_fine_action_enumerator = {
    101: "warm_up_wristankle",
    102: "warm_up_pectoral",
    103: "warm_up_eblowback",
    104: "warm_up_bodylean_right_arm",
    105: "warm_up_bodylean_left_arm",
    106: "warm_up_bow_right",
    107: "warm_up_bow_left",
    201: "walk",
    301: "run",
    401: "jump_handsup",
    402: "jump_vertical",
    501: "drink_bottle_righthand",
    502: "drink_bottle_lefthand",
    503: "drink_cup_righthand",
    504: "drink_cup_lefthand",
    505: "drink_both_hands",
    601: "lift_dumbbell with _right hand",
    602: "lift_dumbbell with _left hand",
    603: "Lift dumbells with both hands",
    604: "lift_dumbbell over head",
    605: "lift_dumbells with both hands and bend legs",
    701: "sit",
    801: "eat_finger_right",
    802: "eat_pie or hamburger",
    803: "Eat with left hand",
    901: "Turn steering wheel",
    1001: "Take out phone, call and put phone back",
    1002: "Call with left hand",
    1101: "boxing_left_right",
    1102: "boxing_left_upwards",
    1103: "boxing_right_upwards",
    1104: "boxing_right_left",
    1201: "throw_right_hand",
    1202: "throw_both_hands",
}

In [9]:
humanact12_kinematic_chain = [[0, 1, 4, 7, 10], [0, 2, 5, 8, 11], [0, 3, 6, 9, 12, 15], [9, 13, 16, 18, 20, 22], [9, 14, 17, 19, 21, 23]]


In [10]:
humanact12_raw_offsets = np.array([[0,0,0],
                               [1,0,0],
                               [-1,0,0],
                               [0,1,0],
                               [0,-1,0],
                               [0,-1,0],
                               [0,1,0],
                               [0,-1,0],
                               [0,-1,0],
                               [0,1,0],
                               [0,0,1],
                               [0,0,1],
                               [0,1,0],
                               [1,0,0],
                               [-1,0,0],
                               [0,0,1],
                               [0,-1,0],
                               [0,-1,0],
                               [0,-1,0],
                               [0,-1,0],
                               [0,-1,0],
                               [0,-1,0],
                               [0,-1,0],
                               [0,-1,0]])

In [13]:
HAT_INV_SKEW_SYMMETRIC_TOL = 1e-5

In [11]:
data_path='/home/youngjoon/Desktop/Dataset/humanact12/'

In [12]:
data_list=os.listdir(data_path)
data_list.sort()
data_list

['P01G01R01F0001T0064A0101.npy',
 'P01G01R01F0069T0143A0102.npy',
 'P01G01R01F0151T0190A0103.npy',
 'P01G01R01F0449T0505A0201.npy',
 'P01G01R01F0718T0825A0301.npy',
 'P01G01R01F0839T0955A0401.npy',
 'P01G01R01F1114T1267A0501.npy',
 'P01G01R01F1316T1416A0502.npy',
 'P01G01R01F1444T1577A0601.npy',
 'P01G01R01F1580T1665A0602.npy',
 'P01G01R01F1683T1740A0603.npy',
 'P01G01R01F1777T1844A0604.npy',
 'P01G01R02F0001T0071A0604.npy',
 'P01G01R02F0084T0117A0601.npy',
 'P01G01R02F0139T0147A0602.npy',
 'P01G01R02F0148T0188A0603.npy',
 'P01G01R02F0209T0272A0604.npy',
 'P01G01R02F0311T0344A0501.npy',
 'P01G01R02F0347T0387A0502.npy',
 'P01G01R02F0462T0667A0401.npy',
 'P01G01R02F0702T0800A0301.npy',
 'P01G01R02F0927T1025A0201.npy',
 'P01G01R02F1077T1165A0101.npy',
 'P01G01R02F1170T1208A0102.npy',
 'P01G01R02F1227T1302A0103.npy',
 'P01G01R02F1370T1406A0104.npy',
 'P01G01R02F1416T1435A0105.npy',
 'P01G01R03F0006T0200A0201.npy',
 'P01G01R03F0216T0292A0301.npy',
 'P01G01R03F0304T0336A0501.npy',
 'P01G01R0

In [15]:
for file_name in data_list:
        full_path = os.path.join(data_path, file_name)
        pose_raw = np.load(full_path)

In [23]:
pose_raw

array([[[-0.40351524,  0.29413164,  2.60027003],
        [-0.34430914,  0.37394949,  2.62289463],
        [-0.46549644,  0.36924302,  2.64690215],
        ...,
        [-0.93375842, -0.344807  ,  2.42676547],
        [-0.42498249,  0.19635575,  2.3647307 ],
        [-0.99995341, -0.38225881,  2.38595918]],

       [[-0.38888806,  0.29199837,  2.60865387],
        [-0.3297524 ,  0.37221287,  2.63002635],
        [-0.45049179,  0.3669211 ,  2.65608145],
        ...,
        [-0.92109958, -0.33509542,  2.43080171],
        [-0.41957843,  0.1939075 ,  2.37008035],
        [-0.98652416, -0.37210704,  2.38838358]],

       [[-0.37552376,  0.28868811,  2.62192748],
        [-0.31708111,  0.36985439,  2.64153686],
        [-0.43678468,  0.36245788,  2.67155704],
        ...,
        [-0.91258655, -0.32877758,  2.43969561],
        [-0.4151814 ,  0.18877771,  2.37987849],
        [-0.97906006, -0.3619663 ,  2.39577015]],

       ...,

       [[-0.50527736,  0.24554421,  2.85509521],
        [-0

In [22]:
offset_mat=np.tile(pose_raw[0,0],(pose_raw.shape[1],1))
offset_mat

array([[-0.40351524,  0.29413164,  2.60027003],
       [-0.40351524,  0.29413164,  2.60027003],
       [-0.40351524,  0.29413164,  2.60027003],
       [-0.40351524,  0.29413164,  2.60027003],
       [-0.40351524,  0.29413164,  2.60027003],
       [-0.40351524,  0.29413164,  2.60027003],
       [-0.40351524,  0.29413164,  2.60027003],
       [-0.40351524,  0.29413164,  2.60027003],
       [-0.40351524,  0.29413164,  2.60027003],
       [-0.40351524,  0.29413164,  2.60027003],
       [-0.40351524,  0.29413164,  2.60027003],
       [-0.40351524,  0.29413164,  2.60027003],
       [-0.40351524,  0.29413164,  2.60027003],
       [-0.40351524,  0.29413164,  2.60027003],
       [-0.40351524,  0.29413164,  2.60027003],
       [-0.40351524,  0.29413164,  2.60027003],
       [-0.40351524,  0.29413164,  2.60027003],
       [-0.40351524,  0.29413164,  2.60027003],
       [-0.40351524,  0.29413164,  2.60027003],
       [-0.40351524,  0.29413164,  2.60027003],
       [-0.40351524,  0.29413164,  2.600

In [26]:
pose_mat=pose_raw-offset_mat
pose_mat.shape

(40, 24, 3)

In [28]:
data=[]
labels=[]
lengths=[]

In [33]:
for file_name in data_list:
        full_path = os.path.join(data_path, file_name)
        pose_raw = np.load(full_path)
        offset_mat=np.tile(pose_raw[0][0],(pose_raw.shape[1],1))
        pose_mat=pose_raw-offset_mat
        label = file_name[file_name.find('A') + 1: file_name.find('.')]
        labels.append(label)
        data.append((pose_mat,label))
        lengths.append(pose_mat.shape[0])
labels.sort()
cumsum=np.cumsum([0]+lengths)
print("Total number of frames {}, videos {}, action types {}".format(cumsum[-1], len(data_list), len(labels)))
label_enc = dict(zip(labels, np.arange(len(labels))))
label_enc_rev = dict(zip(np.arange(len(labels)), labels))        
        

Total number of frames 180198, videos 1191, action types 1191


In [34]:
labels

['0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0102',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 '0103',
 

In [38]:
print(label_enc)
print("-"*50)
# print(label_enc_rev)

{'0101': 24, '0102': 73, '0103': 116, '0104': 142, '0105': 166, '0106': 190, '0107': 214, '0201': 261, '0301': 311, '0401': 365, '0402': 405, '0501': 432, '0502': 475, '0503': 486, '0504': 489, '0505': 493, '0601': 538, '0602': 583, '0603': 630, '0604': 673, '0605': 711, '0701': 765, '0801': 798, '0802': 823, '0803': 842, '0901': 898, '1001': 926, '1002': 959, '1101': 985, '1102': 1024, '1103': 1065, '1104': 1099, '1201': 1152, '1202': 1190}
--------------------------------------------------


In [39]:
label=label_enc[label]

In [40]:
label

1099